# KDSH Dataset Evaluation Notebook
## Narrative Consistency Reasoning Engine on Real Data

This notebook evaluates the consistency engine on the actual KDSH dataset containing:
- **81 training samples** with ground truth labels (consistent/contradict)
- **61 test samples** for prediction
- **Full book narratives** from "In Search of the Castaways" and "The Count of Monte Cristo"

The pipeline will:
1. Load character backstories and their book narratives
2. Run the full 7-step consistency reasoning pipeline
3. Evaluate accuracy against ground truth labels
4. Generate predictions for test set
5. Analyze errors and confidence scores

In [ ]:
import sys
import os
import time
import warnings
import logging
from pathlib import Path


project_root = os.getcwd()
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Configure logging
warnings.filterwarnings("ignore")
if not logging.root.handlers:
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s | %(levelname)s | %(message)s",
        datefmt="%H:%M:%S",
        force=True,
    )
logger = logging.getLogger(__name__)
logging.getLogger("transformers").setLevel(logging.ERROR)
logging.getLogger("pathway").setLevel(logging.ERROR)

# Import libraries
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm

print("=" * 70)
print("ENVIRONMENT CHECK")
print("=" * 70)
print(f"Python Version: {sys.version}")
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
print("=" * 70)

ENVIRONMENT CHECK
Python Version: 3.12.3 (main, Nov  6 2025, 13:44:16) [GCC 13.3.0]
PyTorch Version: 2.5.1+cu121
CUDA Available: True
GPU: NVIDIA GeForce RTX 4070 Laptop GPU
CUDA Version: 12.1


In [ ]:
from utils.gpu_utils import init_gpu, get_gpu_memory_stats

# Initialize GPU 
logger.info("Initializing GPU...")
device = init_gpu()
logger.info(f"Using device: {device}")

if torch.cuda.is_available():
    mem_stats = get_gpu_memory_stats()
    print(f"\nGPU Memory Initialized:")
    print(f"  Allocated: {mem_stats['allocated_gb']:.2f} GB")
    print(f"  Reserved: {mem_stats['reserved_gb']:.2f} GB")
    print(f"  Total: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

08:52:39 | INFO | Initializing GPU...
08:52:39 | INFO | Number of GPUs: 1
08:52:39 | INFO | GPU 0: NVIDIA GeForce RTX 4070 Laptop GPU
08:52:39 | INFO |   Memory: 8.59 GB
08:52:39 | INFO | TF32 precision enabled for faster computation
08:52:39 | INFO | cuDNN benchmark mode enabled
08:52:39 | INFO | Using device: True



GPU Memory Initialized:
  Allocated: 6.77 GB
  Reserved: 6.85 GB
  Total: 8.59 GB


## Section 1: Load and Explore Dataset

In [ ]:
from utils.dataset_loader import DatasetLoader

# Load dataset
loader = DatasetLoader(dataset_dir="./Dataset")

stats = loader.get_statistics()

print("=" * 70)
print("DATASET STATISTICS")
print("=" * 70)
print(f"Train Samples: {stats['train_samples']}")
print(f"Test Samples: {stats['test_samples']}")
print(f"Books: {stats['books']}")
print(f"\nTrain Label Distribution:")
for label, count in stats.get('train_labels', {}).items():
    print(f"  {label}: {count}")
print(f"\nNarrative Sizes (characters):")
for book, size in stats['narrative_sizes'].items():
    print(f"  {book}: {size:,}")
print("=" * 70)

[INFO] Loaded narrative: In search of the castaways (826081 chars)
[INFO] Loaded narrative: The Count of Monte Cristo (2628121 chars)
[INFO] Loaded train data: 80 samples
[INFO] Loaded test data: 60 samples
DATASET STATISTICS
Train Samples: 80
Test Samples: 60
Books: ['In search of the castaways', 'The Count of Monte Cristo']

Train Label Distribution:
  consistent: 51
  contradict: 29

Narrative Sizes (characters):
  In search of the castaways: 826,081
  The Count of Monte Cristo: 2,628,121


In [11]:
# Load train and test data
train_df = loader.load_train_data()
test_df = loader.load_test_data()

print("\nTRAIN DATA SAMPLE:")
print(train_df.head(3))
print(f"\nTrain DataFrame Info:")
print(f"Shape: {train_df.shape}")
print(f"Columns: {list(train_df.columns)}")

print("\n\nTEST DATA SAMPLE:")
print(test_df.head(3))
print(f"\nTest DataFrame Info:")
print(f"Shape: {test_df.shape}")
print(f"Columns: {list(test_df.columns)}")

[INFO] Loaded train data: 80 samples
[INFO] Loaded test data: 60 samples

TRAIN DATA SAMPLE:
    id                   book_name        char  \
0   46  In Search of the Castaways    Thalcave   
1  137   The Count of Monte Cristo       Faria   
2   74  In Search of the Castaways  Kai-Koumou   

                                             caption  \
0                                                NaN   
1  The Origin of His Connection with the Count of...   
2                                                NaN   

                                             content       label  
0  Thalcave’s people faded as colonists advanced;...  consistent  
1  Suspected again in 1815, he was re-arrested an...  contradict  
2  Before each fight he studied the crack-pattern...  consistent  

Train DataFrame Info:
Shape: (80, 6)
Columns: ['id', 'book_name', 'char', 'caption', 'content', 'label']


TEST DATA SAMPLE:
    id                   book_name      char  \
0   95   The Count of Monte Cristo  Noi

## Section 2: Initialize Consistency Engine

In [ ]:
import importlib
import config
config.MODELS["verification"] = "cross-encoder/nli-deberta-v3-base"
import models.verifier
importlib.reload(models.verifier)

from pipeline.consistency_engine import NarrativeConsistencyEngine
from utils.evaluation import ConsistencyEvaluator

# Initialize consistency engine WITHOUT rationale to save VRAM
logger.info("Loading consistency engine models...")
engine = NarrativeConsistencyEngine(enable_rationale=False)
logger.info("Engine ready!")
logger.info(f"Pathway enabled: {engine.retriever.use_pathway} (PATHWAY_AVAILABLE={getattr(engine.retriever, 'use_pathway', False)})")

# Initialize evaluator
evaluator = ConsistencyEvaluator()

print("\n" + "=" * 70)
print("CONSISTENCY ENGINE INITIALIZED (rationale: lazy-loaded on demand)")
print("=" * 70)
print(f"Device: {device}")
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    mem_stats = get_gpu_memory_stats()
    print(f"GPU Memory - Allocated: {mem_stats['allocated_gb']:.2f} GB, Reserved: {mem_stats['reserved_gb']:.2f} GB")
print("Pathway backend: {}".format("Pathway" if engine.retriever.use_pathway else "NumPy fallback"))
print("=" * 70)

[INFO] GPU detected: NVIDIA GeForce RTX 4070 Laptop GPU
[INFO] CUDA Version: 12.1


08:50:59 | INFO | Loading consistency engine models...


[INFO] Initializing claim_extraction model: Qwen/Qwen2.5-3B-Instruct
[INFO] Device: cuda
[INFO] Loading Qwen model...


08:51:03 | INFO | We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

[INFO] Qwen model loaded successfully
[DEBUG] Enabled FP16 precision on GPU
[INFO] Initializing embedding model: sentence-transformers/all-MiniLM-L6-v2
[INFO] Device: cuda
[INFO] Loading embedding model...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Embedding model loaded. Dimension: 384
[INFO] Initializing verification model: cross-encoder/nli-deberta-v3-base
[INFO] Device: cuda
[INFO] Loading DeBERTa verification model...


08:51:18 | INFO | Engine ready!
08:51:18 | INFO | Pathway enabled: True (PATHWAY_AVAILABLE=True)


[INFO] DeBERTa model loaded successfully
[DEBUG] Enabled FP16 precision on GPU

CONSISTENCY ENGINE INITIALIZED (rationale: lazy-loaded on demand)
Device: True
GPU Available: True
GPU Memory - Allocated: 6.77 GB, Reserved: 6.85 GB
Pathway backend: Pathway


In [5]:
status = engine.get_pipeline_status()
print("\nPipeline Status:")
print(f"  Pathway available: {status['pathway_available']}")
print(f"  Retrieval backend: {'Pathway' if status['pathway_available'] else 'NumPy fallback'}")



Pipeline Status:
  Pathway available: True
  Retrieval backend: Pathway


## Section 3: Evaluate on Training Set

The engine will check consistency for each backstory against its book narrative and compare against ground truth labels.

In [ ]:
# Evaluate on training set
num_train_samples = 80  

print(f"\nEvaluating on {num_train_samples} training samples...")
print("=" * 70)

predictions = []
processing_times = []

for idx in tqdm(range(min(num_train_samples, len(train_df))), desc="Train samples"):
    try:

        sample = loader.get_sample(idx, is_train=True)
        
        # Run consistency check
        start_time = time.time()
        result = engine.verify(
            backstory=sample['backstory'],
            narrative=sample['narrative']
        )
        elapsed = time.time() - start_time
        processing_times.append(elapsed)
        
        # Parse labels
        true_label = 1 if sample['label'] == 'consistent' else 0
        pred_label = result.consistency_label
        
        # Calculate confidence score (0.0 to 1.0 based on label and num_contradictions)
        confidence = 1.0 if pred_label == 1 else max(0.0, 1.0 - (result.num_contradictions * 0.1))
        
        # Add to evaluator
        evaluator.add_prediction(
            pred_label=pred_label,
            true_label=true_label,
            sample_id=sample['id'],
            character=sample['character'],
            book=sample['book'],
            confidence=confidence,
            rationale=result.rationale if result.rationale else "No rationale generated"
        )
        
        predictions.append({
            'id': sample['id'],
            'character': sample['character'],
            'predicted': 'consistent' if pred_label else 'contradict',
            'actual': sample['label'],
            'score': confidence,
            'time_ms': elapsed * 1000,
            'match': '✓' if pred_label == true_label else '✗',
        })
        
    except Exception as e:
        logger.error(f"Error on sample {idx}: {str(e)[:100]}")
        continue

pred_df = pd.DataFrame(predictions)
print("\nTRAIN SET PREDICTIONS:")
print(pred_df.to_string())

print(f"\nProcessing Time Stats (ms):")
times_series = pd.Series(processing_times)
print(f"  Mean: {times_series.mean():.1f} ms")
print(f"  Median: {times_series.median():.1f} ms")
print(f"  Max: {times_series.max():.1f} ms")


Evaluating on 80 training samples...


Train samples:   0%|          | 0/80 [00:00<?, ?it/s]

[DEBUG] Extracted 14 claims from backstory
[DEBUG] Split narrative into 318 chunks
[INFO] Indexing 318 narrative chunks...
[INFO] Chunk indexing complete. Embedding shape: (318, 384)


08:52:57 | INFO | Total processing time: 9.84s
08:52:57 | INFO | ============================================================
Train samples:   1%|▏         | 1/80 [00:09<12:57,  9.84s/it]

[DEBUG] Extracted 7 claims from backstory
[DEBUG] Split narrative into 1011 chunks
[INFO] Indexing 1011 narrative chunks...


...

[DEBUG] Extracted 7 claims from backstory
[DEBUG] Split narrative into 1011 chunks
[INFO] Indexing 1011 narrative chunks...
[INFO] Chunk indexing complete. Embedding shape: (1011, 384)


09:00:58 | INFO | Total processing time: 6.02s
09:00:58 | INFO | ============================================================
Train samples:  99%|█████████▉| 79/80 [08:11<00:05,  5.54s/it]

[DEBUG] Extracted 13 claims from backstory
[DEBUG] Split narrative into 318 chunks
[INFO] Indexing 318 narrative chunks...
[INFO] Chunk indexing complete. Embedding shape: (318, 384)


09:01:02 | INFO | Total processing time: 4.36s
09:01:02 | INFO | ============================================================
Train samples: 100%|██████████| 80/80 [08:15<00:00,  6.19s/it]


TRAIN SET PREDICTIONS:
     id             character   predicted      actual  score       time_ms match
0    46              Thalcave  consistent  consistent    1.0   9842.114687     ✓
1   137                 Faria  consistent  contradict    1.0   5380.285025     ✗
2    74            Kai-Koumou  consistent  consistent    1.0   3412.029743     ✓
3   109              Noirtier  consistent  contradict    1.0  34966.070175     ✗
4   104              Noirtier  consistent  consistent    1.0   4260.234833     ✓
5    35  Tom Ayrton/Ben Joyce  contradict  contradict    0.6   3385.541677     ✓
6    18       Jacques Paganel  consistent  consistent    1.0   3851.674557     ✓
7    31  Tom Ayrton/Ben Joyce  consistent  consistent    1.0  13204.327583     ✓
8    68            Kai-Koumou  consistent  consistent    1.0   6432.808876     ✓
9     9       Jacques Paganel  consistent  consistent    1.0   3653.293371     ✓
10   84            Kai-Koumou  contradict  consistent    0.6   3627.358437     ✗
11  

## Section 4: Evaluate Results

In [ ]:
# Get evaluation metrics
metrics = evaluator.get_metrics()

print("\n" + "=" * 70)
print("EVALUATION METRICS")
print("=" * 70)
print(evaluator.get_summary())


print("\n--- PER-BOOK PERFORMANCE ---")
book_analysis = evaluator.analyze_by_book()
for book, book_metrics in book_analysis.items():
    print(f"\n{book}:")
    print(f"  Samples: {book_metrics['samples']}")
    print(f"  Correct: {book_metrics['correct']}/{book_metrics['samples']}")
    print(f"  Accuracy: {book_metrics['accuracy']:.2%}")


print("\n--- CONFIDENCE DISTRIBUTION ---")
conf_analysis = evaluator.get_confidence_analysis()
if 'message' not in conf_analysis:
    for key, val in conf_analysis.items():
        print(f"{key}: {val:.4f}")


# Error analysis
print("\n--- ERRORS ---")
errors = evaluator.get_errors()
if len(errors) > 0:
    print(f"Total Errors: {len(errors)}")
    print("\nIncorrect Predictions:")
    print(errors[['id', 'character', 'book', 'confidence']].to_string())
else:
    print("No errors! Perfect accuracy on this sample!")


EVALUATION METRICS

=== Consistency Model Evaluation ===
Samples: 80
Accuracy: 58.75%
Precision: 62.86%
Recall: 86.27%
F1 Score: 72.73%

Confusion Matrix:
  Predicted:  [Contradict  Consistent]
  Actual
  Contradict: [3, 26]
  Consistent: [7, 44]

Per-Class Metrics:
  Contradict - Precision: 30.00%, Recall: 10.34%
  Consistent - Precision: 62.86%, Recall: 86.27%


--- PER-BOOK PERFORMANCE ---

In Search of the Castaways:
  Samples: 49
  Correct: 35/49
  Accuracy: 71.43%

The Count of Monte Cristo:
  Samples: 31
  Correct: 12/31
  Accuracy: 38.71%

--- CONFIDENCE DISTRIBUTION ---
mean_confidence: 0.9488
std_confidence: 0.1414
mean_correct_confidence: 0.9745
mean_incorrect_confidence: 0.9121
min_confidence: 0.3000
max_confidence: 1.0000

--- ERRORS ---
Total Errors: 33

Incorrect Predictions:
     id             character                        book  confidence
0   137                 Faria   The Count of Monte Cristo         1.0
1   109              Noirtier   The Count of Monte Cristo

## Section 5: Generate Test Set Predictions

In [14]:
num_test_samples = 10

print(f"\nGenerating predictions for {num_test_samples} test samples...")
print("=" * 70)

test_predictions = []

for idx in tqdm(range(min(num_test_samples, len(test_df))), desc="Test samples"):
    try:
        sample = loader.get_sample(idx, is_train=False)
        
        # Run consistency check
        result = engine.verify(
            backstory=sample['backstory'],
            narrative=sample['narrative']
        )
        
        # Calculate confidence score
        pred_label = result.consistency_label
        confidence = 1.0 if pred_label == 1 else max(0.0, 1.0 - (result.num_contradictions * 0.1))
        
        test_predictions.append({
            'id': sample['id'],
            'character': sample['character'],
            'book': sample['book'],
            'prediction': pred_label,  # Binary: 1 = consistent, 0 = contradict
            'confidence': confidence,
            'num_claims': len(result.claims) if result.claims else 0,
        })
        
    except Exception as e:
        logger.error(f"Error on test sample {idx}: {str(e)[:100]}")
        test_predictions.append({
            'id': idx,
            'character': '???',
            'book': '???',
            'prediction': 'error',
            'confidence': 0.0,
            'error': str(e)[:50],
        })
        continue

# Show test predictions
test_pred_df = pd.DataFrame(test_predictions)
print("\nTEST SET PREDICTIONS:")
print(test_pred_df.to_string())

print(f"\n--- PREDICTION DISTRIBUTION ---")
print(test_pred_df['prediction'].value_counts())

print(f"\n--- CONFIDENCE STATISTICS ---")
confidences = test_pred_df[test_pred_df['prediction'] != 'error']['confidence']
print(f"Mean Confidence: {confidences.mean():.4f}")
print(f"Min Confidence: {confidences.min():.4f}")
print(f"Max Confidence: {confidences.max():.4f}")


Generating predictions for 10 test samples...


Test samples:   0%|          | 0/10 [00:00<?, ?it/s]

[DEBUG] Extracted 7 claims from backstory
[DEBUG] Split narrative into 1011 chunks
[INFO] Indexing 1011 narrative chunks...


09:04:28 | INFO | Total processing time: 4.92s
09:04:28 | INFO | ============================================================
Test samples:  10%|█         | 1/10 [00:04<00:44,  4.92s/it]

[INFO] Chunk indexing complete. Embedding shape: (1011, 384)
[DEBUG] Extracted 12 claims from backstory
[DEBUG] Split narrative into 1011 chunks
[INFO] Indexing 1011 narrative chunks...
[INFO] Chunk indexing complete. Embedding shape: (1011, 384)


...

[DEBUG] Extracted 11 claims from backstory
[DEBUG] Split narrative into 1011 chunks
[INFO] Indexing 1011 narrative chunks...
[INFO] Chunk indexing complete. Embedding shape: (1011, 384)


09:05:10 | INFO | Total processing time: 4.87s
09:05:10 | INFO | ============================================================
Test samples:  90%|█████████ | 9/10 [00:47<00:05,  5.08s/it]

[DEBUG] Extracted 7 claims from backstory
[DEBUG] Split narrative into 318 chunks
[INFO] Indexing 318 narrative chunks...


09:05:13 | WARNING |   ⚠ No evidence found for claim: Fletcher had an unstable personal record....
09:05:13 | WARNING |   ⚠ No evidence found for claim: Fletcher had an unstable personal record....
09:05:14 | WARNING |   ⚠ No evidence found for claim: Fletcher advised others about his unstable persona...
09:05:14 | INFO | Total processing time: 3.39s
09:05:14 | INFO | ============================================================
Test samples: 100%|██████████| 10/10 [00:50<00:00,  5.09s/it]

[INFO] Chunk indexing complete. Embedding shape: (318, 384)

TEST SET PREDICTIONS:
    id             character                        book  prediction  confidence  num_claims
0   95              Noirtier   The Count of Monte Cristo           1         1.0           7
1  136                 Faria   The Count of Monte Cristo           0         0.6          12
2   59              Thalcave  In Search of the Castaways           1         1.0          12
3   60              Thalcave  In Search of the Castaways           1         1.0           9
4  124                 Faria   The Count of Monte Cristo           1         1.0           9
5  111              Noirtier   The Count of Monte Cristo           1         1.0           7
6  135                 Faria   The Count of Monte Cristo           0         0.7           9
7   27  Tom Ayrton/Ben Joyce  In Search of the Castaways           1         1.0           9
8  110              Noirtier   The Count of Monte Cristo           0         0.6

## Structured Dossier Output 

Generate academic-grade dossiers with explicit claim–excerpt pairs .

In [ ]:
# Load source texts for attribution
from pipeline.dossier_utils import load_source_books, generate_structured_output

source_book_paths = load_source_books("./Dataset")
print(f"\n✓ Loaded {len(source_book_paths)} source books for exact attribution:")
for book, path in source_book_paths.items():
    print(f"  - {book}: {path}")

# Initialize dossier 
engine.dossier_builder.load_source_texts(source_book_paths)
print("\n✓ Dossier builder initialized with source texts")

09:06:11 | INFO | Found source text: In search of the castaways
09:06:11 | INFO | Found source text: The Count of Monte Cristo
09:06:11 | INFO | ✓ Loaded In search of the castaways (826,131 chars)
09:06:11 | INFO | ✓ Loaded The Count of Monte Cristo (2,646,614 chars)



✓ Loaded 2 source books for exact attribution:
  - In search of the castaways: ./Dataset/Books/In search of the castaways.txt
  - The Count of Monte Cristo: ./Dataset/Books/The Count of Monte Cristo.txt

✓ Dossier builder initialized with source texts


In [ ]:
# Enable rationale generator for detailed dossier generation
# (First use will trigger lazy loading)
print("\n" + "="*80)
print("GENERATING STRUCTURED DOSSIERS (Priority 1: Explicit Claim-Excerpt Pairs)")
print("This will trigger lazy-loading of rationale generator on first use...")
print("="*80)


engine.enable_rationale = True

structured_dossiers = []
structured_output = []

for idx in range(min(3, len(test_df))):  # Sample 3 for demonstration
    try:
        sample = loader.get_sample(idx, is_train=False)
        
        print(f"\n[{idx+1}/3] Processing {sample['character']} ({sample['book']})...")
        
        
        dossier_dict, dossier_obj = engine.build_structured_dossier(
            character_id=sample['id'],
            character_name=sample['character'],
            book=sample['book'],
            backstory=sample['backstory'],
            narrative=sample['narrative'],
            source_book_paths=source_book_paths,
        )
        
        structured_dossiers.append(dossier_dict)
        
        
        output_row = generate_structured_output(
            dossier_dict,
            engine,
            sample['id'],
            sample,
        )
        structured_output.append(output_row)
        
        # Display key results
        print(f"  ✓ Consistency: {'CONSISTENT (1)' if dossier_dict['overall_consistency'] == 1 else 'CONTRADICTED (0)'}")
        print(f"  ✓ Claims: {dossier_dict['statistics']['total_claims']} total, "
              f"{dossier_dict['statistics']['contradicted_claims']} contradicted")
        print(f"  ✓ Executive Summary: {dossier_dict['executive_summary'][:100]}...")
        
        # Display structured claim-excerpt pair
        if dossier_dict['claim_evidence_pairs']:
            first_pair = dossier_dict['claim_evidence_pairs'][0]
            print(f"\n  STRUCTURED EVIDENCE LINKAGE:")
            print(f"    CLAIM: {first_pair['claim_text'][:80]}...")
            if first_pair['evidence']:
                print(f"    EXCERPT: {first_pair['evidence']['excerpt'][:80]}...")
                print(f"    SOURCE: {first_pair['evidence']['source'][:80]}...")
            print(f"    ANALYSIS: {first_pair['constraint_analysis'][:100]}...")
        
    except Exception as e:
        logger.error(f"Error processing sample {idx}: {str(e)[:100]}")
        continue

print(f"\n✓ Generated {len(structured_dossiers)} structured dossiers with explicit linkage")

09:06:13 | INFO | Building structured dossier for Noirtier...
09:06:13 | INFO | ✓ Loaded In search of the castaways (826,131 chars)
09:06:13 | INFO | ✓ Loaded The Count of Monte Cristo (2,646,614 chars)



GENERATING STRUCTURED DOSSIERS (Priority 1: Explicit Claim-Excerpt Pairs)
This will trigger lazy-loading of rationale generator on first use...

[1/3] Processing Noirtier (The Count of Monte Cristo)...
[DEBUG] Extracted 9 claims from backstory
[DEBUG] Split narrative into 1011 chunks
[INFO] Indexing 1011 narrative chunks...
[INFO] Chunk indexing complete. Embedding shape: (1011, 384)


09:06:18 | INFO | Total processing time: 4.94s
09:06:18 | INFO | ============================================================
09:06:18 | INFO | ✓ Dossier built: 9 claims, 2 contradicted
09:06:18 | INFO | Building structured dossier for Faria...
09:06:18 | INFO | ✓ Loaded In search of the castaways (826,131 chars)
09:06:18 | INFO | ✓ Loaded The Count of Monte Cristo (2,646,614 chars)


  ✓ Consistency: CONSISTENT (1)
  ✓ Claims: 9 total, 2 contradicted
  ✓ Executive Summary: Noirtier's backstory is CONSISTENT with the narrative. Of 9 claims extracted, 7 are supported and 2 ...

  STRUCTURED EVIDENCE LINKAGE:
    CLAIM: Noirtier handed a dossier to a British spy....
    EXCERPT: y to the Comte de Salvieux that I would like to see him.”

“Now, then, go,” said...
    SOURCE: Line ~4004, Para ~1009 | Context: 'in person.”

“A thousand thanks—and now for t...
    ANALYSIS: CLAIM SUPPORTED: The claim 'Noirtier handed a dossier to a British spy....' is directly supported by...

[2/3] Processing Faria (The Count of Monte Cristo)...
[DEBUG] Extracted 18 claims from backstory
[DEBUG] Split narrative into 1011 chunks
[INFO] Indexing 1011 narrative chunks...


09:06:32 | WARNING |   ⚠ No evidence found for claim: The work was on human intelligence....


...

  ✓ Consistency: CONTRADICTED (0)
  ✓ Claims: 18 total, 8 contradicted
  ✓ Executive Summary: Faria's backstory is INCONSISTENT with the narrative. Of 18 claims, 8 significant contradictions wer...

  STRUCTURED EVIDENCE LINKAGE:
    CLAIM: He lived on an island from 1800 onward....
    EXCERPT: s.”

“The story begins in 1815.”

“Ah,” said Monte Cristo, “1815 is not yesterda...
    SOURCE: Line ~23593, Para ~5664 | Context: 'ear of tiring your excellency.”

“Go on, M. ...
    ANALYSIS: CONSTRAINT VIOLATED: The claim states 'He lived on an island from 1800 onward....' However, the narr...

[3/3] Processing Thalcave (In Search of the Castaways)...
[DEBUG] Extracted 12 claims from backstory
[DEBUG] Split narrative into 318 chunks
[INFO] Indexing 318 narrative chunks...


09:06:38 | WARNING |   ⚠ No evidence found for claim: He posed as a relay-station hand....


[INFO] Chunk indexing complete. Embedding shape: (318, 384)


09:06:39 | WARNING |   ⚠ No evidence found for claim: He posed as a relay-station hand....
09:06:39 | INFO | Total processing time: 6.00s
09:06:39 | INFO | ============================================================
09:06:39 | WARNING | No verification found for claim: He posed as a relay-station hand....
09:06:39 | WARNING | No verification found for claim: He posed as a relay-station hand....
09:06:39 | INFO | ✓ Dossier built: 12 claims, 0 contradicted


  ✓ Consistency: CONSISTENT (1)
  ✓ Claims: 12 total, 0 contradicted
  ✓ Executive Summary: Thalcave's backstory is CONSISTENT with the narrative. Of 12 claims extracted, 10 are supported and ...

  STRUCTURED EVIDENCE LINKAGE:
    CLAIM: He slipped captivity....
    ANALYSIS: CLAIM SUPPORTED: The claim 'He slipped captivity....' is directly supported by narrative evidence: '...

✓ Generated 3 structured dossiers with explicit linkage


In [17]:
import json
import numpy as np

class NumpySafeEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super().default(obj)


In [ ]:
# Save structured output to CSV 
import pandas as pd

if structured_output:
    structured_df = pd.DataFrame(structured_output)
    
    print("\n" + "="*80)
    print("STRUCTURED OUTPUT WITH EXPLICIT LINKAGE")
    print("="*80)
    print(structured_df[['character', 'prediction', 'claim', 'excerpt', 'analysis']].to_string())
    
    
    structured_df.to_csv('./structured_dossier_output.csv', index=False)
    print(f"\n✓ Saved structured dossier output to: structured_dossier_output.csv")
    print(f"  Columns: {structured_df.columns.tolist()}")
    print(f"  Format: Explicit claim-excerpt pairs with constraint analysis")
    
    # Also save full dossiers as JSON for detailed reference
    with open('./dossiers_detailed.json', 'w') as f:
        json.dump(structured_dossiers, f, indent=2, cls=NumpySafeEncoder)
    print(f"✓ Saved detailed dossiers to: dossiers_detailed.json")
else:
    print("No structured output generated yet")


STRUCTURED OUTPUT WITH EXPLICIT LINKAGE
  character  prediction                                                  claim                                                                                                                                                                                                                                                                                                                excerpt                                                                                                                                                                                                                                                                                                                                                                                      analysis
0  Noirtier           1  The dossier contained information about a conspiracy.      Oh, no,” replied Caderousse, “that I can answer for, you did not. I\nonly wish I could see it now as pl

## Section 6: Detailed Analysis - Sample Cases

In [ ]:
print("\n" + "=" * 70)
print("EXAMPLE 1: CONSISTENT BACKSTORY")
print("=" * 70)

consistent_samples = loader.get_consistent_samples(num=1)
if consistent_samples:
    sample = consistent_samples[0]
    print(f"\nCharacter: {sample['character']}")
    print(f"Book: {sample['book']}")
    print(f"Label: CONSISTENT")
    print(f"\nBackstory ({len(sample['backstory'])} chars):")
    print(sample['backstory'][:500] + "..." if len(sample['backstory']) > 500 else sample['backstory'])
    
    
    result = engine.verify(
        backstory=sample['backstory'],
        narrative=sample['narrative']
    )
    
    print(f"\nConsistency Engine Results:")
    print(f"  Consistency Label: {result.consistency_label} (1=consistent, 0=inconsistent)")
    print(f"  Contradictions Found: {result.num_contradictions}")
    print(f"  Rationale: {result.rationale if result.rationale else 'None generated'}")
    
    if result.verifications:
        print(f"\n  Claim Verifications ({len(result.verifications)} total):")
        for i, v in enumerate(result.verifications[:3]):  # Show first 3
            print(f"    {i+1}. {v.get('claim', 'N/A')[:60]}...")
            print(f"       Label: {v.get('label', 'N/A')} | Score: {v.get('score', 0):.2f}")


print("\n\n" + "=" * 70)
print("EXAMPLE 2: CONTRADICTING BACKSTORY")
print("=" * 70)

contradict_samples = loader.get_contradict_samples(num=1)
if contradict_samples:
    sample = contradict_samples[0]
    print(f"\nCharacter: {sample['character']}")
    print(f"Book: {sample['book']}")
    print(f"Label: CONTRADICT")
    print(f"\nBackstory ({len(sample['backstory'])} chars):")
    print(sample['backstory'][:500] + "..." if len(sample['backstory']) > 500 else sample['backstory'])
    
    
    result = engine.verify(
        backstory=sample['backstory'],
        narrative=sample['narrative']
    )
    
    print(f"\nConsistency Engine Results:")
    print(f"  Consistency Label: {result.consistency_label} (1=consistent, 0=inconsistent)")
    print(f"  Contradictions Found: {result.num_contradictions}")
    print(f"  Rationale: {result.rationale if result.rationale else 'None generated'}")
    
    if result.verifications:
        print(f"\n  Claim Verifications ({len(result.verifications)} total):")
        for i, v in enumerate(result.verifications[:3]):  # Show first 3
            print(f"    {i+1}. {v.get('claim', 'N/A')[:60]}...")
            print(f"       Label: {v.get('label', 'N/A')} | Score: {v.get('score', 0):.2f}")


EXAMPLE 1: CONSISTENT BACKSTORY

Character: Thalcave
Book: In Search of the Castaways
Label: CONSISTENT

Backstory (273 chars):
Thalcave’s people faded as colonists advanced; his father, last of the tribal guides, knew the pampas geography and animal ways, while his mother died giving birth. Boyhood was spent roaming the plains with his father, learning to track, tame horses and steer by the stars.
[DEBUG] Extracted 13 claims from backstory
[DEBUG] Split narrative into 318 chunks
[INFO] Indexing 318 narrative chunks...
[INFO] Chunk indexing complete. Embedding shape: (318, 384)


09:07:42 | INFO | [LAZY-LOAD] Loading rationale generator on first use...


[INFO] Initializing rationale_generation model: Qwen/Qwen2.5-3B-Instruct
[INFO] Device: cuda
[INFO] Loading Qwen rationale generation model...


09:07:43 | INFO | We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
09:07:43 | INFO | Based on the current allocation process, no modules could be assigned to the following devices due to insufficient memory:
  - 0: 776483968 bytes required
These minimum requirements are specific to this allocation attempt and may vary. Consider increasing the available memory for these devices to at least the specified minimum, or adjusting the model config.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[INFO] Qwen rationale model loaded successfully
[DEBUG] Enabled FP16 precision on GPU


09:10:58 | INFO |   ✓ Rationale generated
09:10:58 | INFO | Total processing time: 202.62s
09:10:58 | INFO | ============================================================



Consistency Engine Results:
  Consistency Label: 1 (1=consistent, 0=inconsistent)
  Contradictions Found: 0
  Rationale: I need to write a scene where Thalcave is in a situation where he needs to use his knowledge of the pampa. The pampa is vast and dangerous, so I want to make sure that Thalcave's knowledge is crucial to surviving this situation.

  Claim Verifications (13 total):
    1. Thalcave's people experienced a decline as European settlers...
       Label: ENTAILMENT | Score: 1.00
    2. Thalcave's father was the last member of the tribe who serve...
       Label: ENTAILMENT | Score: 1.00
    3. Thalcave's mother passed away during childbirth....
       Label: ENTAILMENT | Score: 1.00


EXAMPLE 2: CONTRADICTING BACKSTORY

Character: Faria
Book: The Count of Monte Cristo
Label: CONTRADICT

Backstory (96 chars):
Suspected again in 1815, he was re-arrested and shipped to the Château d’If, this time for life.
[DEBUG] Extracted 7 claims from backstory
[DEBUG] Split narrative into 

09:13:34 | INFO |   ✓ Rationale generated
09:13:34 | INFO | Total processing time: 156.50s
09:13:34 | INFO | ============================================================



Consistency Engine Results:
  Consistency Label: 1 (1=consistent, 0=inconsistent)
  Contradictions Found: 1
  Rationale: - The sentence structure is awkward.
- "Suspected" is a verb, while "re-arrested" is a past participle. They should be consistent.
- "Shipped" is not the correct term for imprisonment; "sentenced" would be more appropriate.
- "Château d'If" is the co

  Claim Verifications (7 total):
    1. In 1815, the person was suspected of a crime....
       Label: NEUTRAL | Score: 0.98
    2. The person was re-arrested in 1815....
       Label: ENTAILMENT | Score: 0.86
    3. The person was sent to the Château d'If in 1815....
       Label: ENTAILMENT | Score: 0.60


## Section 7: Save Results

In [ ]:
# Create final submission results.csv with all required fields
print("\n" + "=" * 80)
print("GENERATING FINAL SUBMISSION: results.csv")
print("=" * 80)

# Convert binary predictions to label format
results_df = test_pred_df[[
    "id",
    "character",
    "book",
    "prediction",
    "confidence",
    "num_claims"
]].copy()

# Add readable label column (0=contradict, 1=consistent)
results_df['label'] = results_df['prediction'].apply(
    lambda x: 'consistent' if x == 1 else ('contradict' if x == 0 else 'unknown')
)

# Reorder columns for final CSV
final_results_df = results_df[[
    "id",
    "character",
    "book",
    "prediction",
    "label",
    "confidence",
    "num_claims"
]].copy()

# Round confidence to 4 decimal places for clean output
final_results_df['confidence'] = final_results_df['confidence'].round(4)

# Convert prediction to int where possible (not 'error')
final_results_df['prediction'] = final_results_df['prediction'].apply(
    lambda x: int(x) if isinstance(x, (int, np.integer, float)) else x
)

# Save to results.csv
final_results_df.to_csv("results.csv", index=False)

print(f"\n✓ results.csv created successfully")
print(f"  Location: ./results.csv")
print(f"  Total rows: {len(final_results_df)}")
print(f"\n✓ CSV Schema (columns in order):")
for i, col in enumerate(final_results_df.columns, 1):
    print(f"  {i}. {col:<15} - {final_results_df[col].dtype}")

print(f"\n✓ Data Summary:")
print(f"  Consistent predictions (1): {(final_results_df['prediction'] == 1).sum()}")
print(f"  Contradict predictions (0): {(final_results_df['prediction'] == 0).sum()}")
print(f"  Mean Confidence: {final_results_df['confidence'].mean():.4f}")
print(f"  Min Confidence: {final_results_df['confidence'].min():.4f}")
print(f"  Max Confidence: {final_results_df['confidence'].max():.4f}")

print(f"\n✓ Sample Output (first 5 rows):")
print(final_results_df.head(5).to_string())
print("\n" + "=" * 80)

In [21]:
# ============================================================================
# FINAL SUMMARY — KDSH Narrative Consistency Reasoning Engine
# ============================================================================

